In [1]:
import os 
os.chdir("../")

In [2]:
%pwd

'c:\\datascience\\mypractice\\Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: str
    

In [6]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_validation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )


        return data_validation_config

In [9]:
import os
from TextSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [12]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):

        prefix = "summarize: "
        # Tokenize the dialogue from the example batch
        inputs = [prefix + doc for doc in example_batch['dialogue']]
        model_inputs = self.tokenizer(
            inputs,                     # Extract dialogue from the example batch
            max_length=1024,             # Set maximum sequence length for input
            truncation=True             # Truncate sequences that exceed max_length
        )

        # Tokenize the summary from the example batch as target
        with self.tokenizer.as_target_tokenizer():
            labels = self.tokenizer(
                example_batch['summary'],  # Extract summary from the example batch
                max_length=128,             # Set maximum sequence length for summary
                truncation=True            # Truncate sequences that exceed max_length
            )

        
        model_inputs["labels"] = labels["input_ids"]
        # Return a dictionary containing input and target encodings
        return model_inputs
    
    def convert(self):
        dataset = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset.map(
            self.convert_examples_to_features,
            batched=True,
        )

        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum"))





In [13]:
try:
    config= ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation =  DataTransformation(config = data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[ 2024-05-01 04:02:22,697 ] 31 TextSummarizer.logging - INFO - yaml file: config\config.yaml loaded successfully
[ 2024-05-01 04:02:22,698 ] 31 TextSummarizer.logging - INFO - yaml file: params.yaml loaded successfully
[ 2024-05-01 04:02:22,699 ] 53 TextSummarizer.logging - INFO - created directory at: artifacts
[ 2024-05-01 04:02:22,699 ] 53 TextSummarizer.logging - INFO - created directory at: artifacts/data_transformation


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\priks\anaconda3\envs\text-summarizer\lib\site-packages\transformers\tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 136359.47 examples/s]
